In [1]:
serp_api = '2db975c7f72d37a545504fb166033d19ccbadcf5eeaf17b56882f9a81d0dafe7'
cohere_api = '6rIWhaa4vGVQKRnSJ9ClSKkgsxQp8EabiXBLH2zl'
elsevier_apikey = 'ce04ffa20c7ee4560e4ffd4761bbe157'



In [2]:
# python cli_demo.py --cohere_api_key 6rIWhaa4vGVQKRnSJ9ClSKkgsxQp8EabiXBLH2zl

In [3]:
import requests
from bs4 import BeautifulSoup
import bs4
import time
import lxml
import cohere
from cohere.classify import Example 
import pandas as pd
from unidecode import unidecode
import json
from qa.search import embedding_search, get_results_paragraphs_multi_process, get_results_paragraphs_from_paper
from qa.bot import GroundedQaBot

In [3]:
class paper_parser():
    def __init__(self, query, apikey, count, start, sort, view, timeout):
        self.query = query
        self.apikey = apikey
        self.count = count
        self.start = start
        self.sort = sort
        self.view = view
        self.timeout = timeout
        self.dic = self.search_for_papers(self.query, self.apikey, self.count, self.start, self.sort, self.view)
        self.piis = list(self.dic.values())
        self.paragraphs = self.get_all_paragraphs()
        self.titles = self.get_all_titles()
        self.authors = self.get_all_1st_authors()
        self.organizations = self.get_all_organizations()
    def search_for_papers(self, query, apikey, count, start, sort, view):
        url = 'https://api.elsevier.com/content/search/sciencedirect?'
        headers = {'X-ELS-APIKey': apikey, 'Accept': 'application/json'}
        params = {'query': query, 'count': count, 'start': start, 'sort': sort, 'view': view}
        r = requests.get(url, headers=headers, params=params)
        json_resp = json.loads(r.text)
        titles, piis, authors = [], [], []
        dic = {}
        for paper in json_resp['search-results']['entry']:
            titles.append(paper['dc:title'])
            piis.append(paper['pii'])
            dic[paper['dc:title']] = paper['pii']
        return dic
    def get_response(self, pii):
        url = f'https://api.elsevier.com/content/article/pii/{pii}'
        response = requests.get(url, headers={"X-ELS-APIKey":self.apikey, "content-type": "text/xml;charset=UTF-8"}, timeout = self.timeout)
        soup = BeautifulSoup(response.content, "xml")
        return soup
    def get_paragraphs(self, pii):
        url = f'https://api.elsevier.com/content/article/pii/{pii}'
        response = requests.get(url, headers={"X-ELS-APIKey":self.apikey, "content-type": "text/xml;charset=UTF-8"}, timeout = self.timeout)
        soup = BeautifulSoup(response.content, "xml")
        abstract = [unidecode(''.join(s.findAll(text=True)).replace('\n', '')) for s in soup.find_all('ce:abstract')][0]
        text = ["Abstract: " + abstract]+[unidecode(''.join(s.findAll(text=True)).replace('\n', '')) for s in soup.find_all('ce:para')]
        return text
    def get_all_1st_authors(self):
        authors = []
        for pii in self.piis:
            url = f'https://api.elsevier.com/content/article/pii/{pii}'
            response = requests.get(url, headers={"X-ELS-APIKey":self.apikey, "content-type": "text/xml;charset=UTF-8"}, timeout = self.timeout)
            soup = BeautifulSoup(response.content, "xml")
            first_name = [''.join(s.findAll(text=True)) for s in soup.find_all('ce:given-name')][0]
            last_name = [''.join(s.findAll(text=True)) for s in soup.find_all('ce:surname')][0]
            name = first_name + ' ' + last_name
            authors.append(name)
        return authors
    def get_all_organizations(self):
        organizations = []
        for pii in self.piis:
            url = f'https://api.elsevier.com/content/article/pii/{pii}'
            response = requests.get(url, headers={"X-ELS-APIKey":self.apikey, "content-type": "text/xml;charset=UTF-8"}, timeout = self.timeout)
            soup = BeautifulSoup(response.content, "xml")            
            organization = [''.join(s.findAll(text=True)) for s in soup.find_all('sa:organization')]
            # join all organizations into one string and split by comma
            organization = ', '.join(organization)
            organizations.append(organization)
        return organizations
    def get_all_paragraphs(self):
        paragraphs = []
        for pii in self.piis:
            paragraphs.append(self.get_paragraphs(pii))
        return paragraphs
    def get_all_titles(self):
        return list(self.dic.keys())
    def get_text(self, pii, apikey, timeout):
        url = f'https://api.elsevier.com/content/article/pii/{pii}'
        response = requests.get(url, headers={"X-ELS-APIKey":apikey, "content-type": "text/xml;charset=UTF-8"}, timeout = timeout)
        soup = BeautifulSoup(response.content, "xml")
        return soup

In [6]:
pp = paper_parser('Machine learning assisted design of FeCoNiCrMn high-entropy alloys with ultra-low hydrogen diffusion coefficients', elsevier_apikey, 1, 0, 'relevance', 'COMPLETE', 60)

In [7]:
example_papers = pp.get_all_paragraphs()
example_titles = pp.get_all_titles()
example_authors = pp.get_all_1st_authors()
example_organizations = pp.get_all_organizations()
example_soup = pp.get_response(pp.piis[0])

In [10]:
soup

<?xml version="1.0" encoding="utf-8"?>
<full-text-retrieval-response xmlns="http://www.elsevier.com/xml/svapi/article/dtd" xmlns:bk="http://www.elsevier.com/xml/bk/dtd" xmlns:cals="http://www.elsevier.com/xml/common/cals/dtd" xmlns:ce="http://www.elsevier.com/xml/common/dtd" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:ja="http://www.elsevier.com/xml/ja/dtd" xmlns:mml="http://www.w3.org/1998/Math/MathML" xmlns:prism="http://prismstandard.org/namespaces/basic/2.0/" xmlns:sa="http://www.elsevier.com/xml/common/struct-aff/dtd" xmlns:sb="http://www.elsevier.com/xml/common/struct-bib/dtd" xmlns:tb="http://www.elsevier.com/xml/common/table/dtd" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xocs="http://www.elsevier.com/xml/xocs/dtd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"><coredata><prism:url>https://api.elsevier.com/content/article/pii/S1359645421009137</prism:url><dc:identifier>doi:10.1016/j.actamat.2021.117535</dc:identifier>

In [9]:
soup = example_soup
[unidecode(''.join(s.findAll(text=True)).replace('\n', '')) for s in soup.find_all('ce:para')]

['The demand for hydrogen (H) energy is rapidly growing in the current society [1-3]. However, the wide application of H energy faces great challenges in the storage and transportation of H gas for that the presence of mobile H atoms in metals could render serious deterioration in the mechanical properties of metals, especially ductility and durability, inducing the so-call hydrogen embrittlement (HE). The mobile H atoms are prone to accumulate at crystal defects i.e., grain boundaries [4,5], interfaces [6-8], and micro voids [9,10], inducing decohesion or stress concentration, and consequently crack initiation and propagation. One common strategy to increase the HE resistance of metals is to suppress severe H accumulation at defects by slowing down H diffusion through the dispersion of irreversible traps like precipitates [11] or reversible traps like grain boundaries [12] and dislocation walls [13]. However, to trap enough diffusive H, the defects should have sufficient density and p

In [38]:
example_papers

[['Abstract: The broad compositional space of high entropy alloys (HEA) is conducive to the design of HEAs with targeted performance. Herein, a data-driven and machine learning (ML) assisted prediction and optimization strategy is proposed to explore the prototype FeCoNiCrMn HEAs with low hydrogen diffusion coefficients. The model for predicting hydrogen solution energies from local HEA chemical environments was constructed via ML algorithms. Based on the inferred correlation between atomic structures and diffusion coefficients of HEAs built using ML models and kinetic Monte Carlo simulations, we employed the whale optimization algorithm to explore HEA atomic structures with low hydrogen diffusion coefficients. HEAs with low H diffusion coefficients were found to have high Co and Mn content. Finally, a quantitative relationship between the diffusion coefficient and chemical composition is proposed to guide the design of HEAs with low H diffusion coefficients and thus strong resistance 

In [12]:
pp.piis

['S1359645421009137']

In [40]:
# save to a csv file
df = pd.DataFrame(example_papers[0], columns = ['paragraphs'])
df['titles'] = example_titles[0]
df['authors'] = example_authors[0]
df['organizations'] = example_organizations[0]
df['piis'] = pp.piis[0]
# df.to_csv('S1359645421009137.csv')

In [3]:
def get_relevant_paper_from_question(question):
    co = cohere.Client('6rIWhaa4vGVQKRnSJ9ClSKkgsxQp8EabiXBLH2zl') 
    response = co.generate( 
        model='command-xlarge-20221108', 
        prompt='Extract the key words and seperate them by commas from the following question:\n\nQuestion: '+ question +'\n\nKey words:', 
        max_tokens=50, 
        temperature=0.6, 
        k=0, 
        p=1, 
        frequency_penalty=0, 
        presence_penalty=0, 
        stop_sequences=["--"], 
        return_likelihoods='NONE') 
    key_words = response.generations[0].text
    print(key_words)
    pp = paper_parser(key_words, elsevier_apikey, 1, 0, 'relevance', 'COMPLETE', 60)
    paragraphs = []
    paragraph_sources = []
    for i in range(len(pp.get_all_paragraphs()[0])):
        paragraphs += pp.get_all_paragraphs()[0][i]
        paragraph_sources += [pp.piis(0)]  * len(url_paragraphs[i])
    return paragraphs, paragraph_sources

In [4]:
get_relevant_paper_from_question('What is the best way to synthesize a membrane with NIPS?')


membrane,NIPS,synthesize


NameError: name 'paper_parser' is not defined

In [4]:
df = pd.read_csv('S1359645421009137'+'.csv')

In [4]:
get_results_paragraphs_from_paper('S1359645421009137')[0]

['The demand for hydrogen (H) energy is rapidly growing in the current society [1-3]. However, the wide application of H energy faces great challenges in the storage and transportation of H gas for that the presence of mobile H atoms in metals could render serious deterioration in the mechanical properties of metals, especially ductility and durability, inducing the so-call hydrogen embrittlement (HE). The mobile H atoms are prone to accumulate at crystal defects i.e., grain boundaries [4,5], interfaces [6-8], and micro voids [9,10], inducing decohesion or stress concentration, and consequently crack initiation and propagation. One common strategy to increase the HE resistance of metals is to suppress severe H accumulation at defects by slowing down H diffusion through the dispersion of irreversible traps like precipitates [11] or reversible traps like grain boundaries [12] and dislocation walls [13]. However, to trap enough diffusive H, the defects should have sufficient density and p

In [4]:
question_list = []
with open("question_list_S1359645421009137.txt", "r") as questions:
    for q in questions:
        q = q.rstrip('\n')
        question_list.append(q)
question_list

['Can you give a summary of the paper?',
 'What’s the motivation?',
 'What’s the objective?',
 'What are the key findings?',
 'What’s hydrogen embrittlement?',
 'What is the alloy system considered?',
 'What are the compositions considered?',
 'How to increase the hydrogen embrittlement resistance?',
 'What are the underlying mechanisms of hydrogen embrittlement resistance in high-entropy alloys?',
 'Describe how the authors performed their study.',
 'What are the computational methods used in this study?',
 'What are the experimental methods used in this study?',
 'What are the synthesis methods used in this study?',
 'What technique is used to determine the saddle point configuration?',
 'What is the definition of the hydrogen solution energy?',
 'Describe the detailed DFT settings.',
 'What is the density of the k point grid used in DFT calculations?',
 'What’s the WOA method?',
 'What does WOA stand for?',
 'What is the objective of using the WOA method?',
 'How is the H diffusion 

In [6]:
bot = GroundedQaBot(cohere_api)
with open('output__S1359645421009137.txt', 'w') as file:
    for question in question_list:
        reply, source_urls, source_texts = bot.answer(question, n_paragraphs=3)
        sources_str = "\n".join(list(set(source_texts)))
        reply_incl_sources = f"{reply}\nSource:\n{sources_str}\n"#pii:\n{source_urls}"
        file.write("question: " + question + '\n' + "answer: " + reply_incl_sources+ '\n')


In [8]:
# Open the file for writing
